In [1]:
import pickle
from sklearn.ensemble import RandomForestClassifier
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from bayes_opt import BayesianOptimization

from general_functions import create_balanced_dataset

In [2]:
file = open("dataset/zone_4.pickle", "rb")
zone_4 = pickle.load(file)

file = open("dataset/zone_7.pickle", "rb")
zone_7 = pickle.load(file)
file = None

In [3]:
#zone_7_resampled = create_balanced_dataset(["dataset/zone_7.pickle"])

#with open("dataset/zone_7_resampled.pickle", "wb") as file:
#    pickle.dump(zone_7_resampled, file)

In [4]:
with open("dataset/zone_4_resampled.pickle", "rb") as file:
    zone_4_resampled = pickle.load(file)
with open("dataset/zone_7_resampled.pickle", "rb") as file:
    zone_7_resampled = pickle.load(file)

In [5]:
zone_4.shape

(7852140, 82)

In [6]:
experiment_arr = [(zone_4_resampled, zone_7), (zone_7_resampled, zone_4)]

In [20]:
most_important_features = zone_4.columns.tolist()[1:]
for num_features in [81, 50]: # [81, 50, 35, 25, 20, 15, 10]
    y_test_all = []
    pred_all = []
    
    feature_importances = {i:0 for i in most_important_features}
    for (training_dataset, test_dataset) in experiment_arr:

        X_train = training_dataset.loc[:, training_dataset.columns != "label_3m"]
        y_train = training_dataset["label_3m"]
        training_dataset = None

        clf = RandomForestClassifier(n_estimators=1, class_weight="balanced", max_features=25, n_jobs=-1)

        clf.fit(X_train, y_train)

        X_test = test_dataset.loc[:, test_dataset.columns != "label_3m"]
        y_test = test_dataset["label_3m"]

        pred = clf.predict(X_test)

        for i, pred_var in enumerate(pred):
            y_test_all.append(y_test[i])
            pred_all.append(pred_var)

        importances = clf.feature_importances_

        
        for i, importance in enumerate(importances):
            feature_importances[most_important_features[i]] += importance
        
    for importance_name in most_important_features:
        feature_importances[importance_name] /= 2
    cohen_score = cohen_kappa_score(y_test_all, pred_all)
    print(feature_importances)
    print("-----------------------")
    print("Cohen's kappa score        ", cohen_score)
    print("\n------------------------------------------------------\n")

{'hpmf_raw': 0.0038683282581401376, 'skyview_raw': 0.004451568570564659, 'impundment_raw': 0.003938460207940128, 'slope_raw': 0.0057675119300079035, 'DEM_ditch_detection': 0.0018079561470795758, 'DEM_ditch_detection_no_streams': 0.002125134644307436, 'conic_mean': 0.0047954733033214025, 'skyview_non_ditch': 0.017280068012418087, 'skyview_gabor': 0.013691250074689836, 'conic_mean_no_streams': 0.0044128676335685995, 'skyview_gabor_no_streams': 0.012521749419490396, 'skyview_mean_2': 0.00533520283174619, 'skyview_mean_3': 0.005080650479185579, 'skyview_mean_4': 0.004293189879394133, 'skyview_mean_6': 0.007137227365554118, 'skyview_median_2': 0.004914472153810017, 'skyview_median_4': 0.005915749202097467, 'skyview_median_6': 0.01009957876534004, 'skyview_min_2': 0.005359683172195564, 'skyview_min_4': 0.0070039130561082245, 'skyview_min_6': 0.009066526147627735, 'skyview_max_2': 0.008579224054014616, 'skyview_max_4': 0.012666519872846704, 'skyview_max_6': 0.017288854401862102, 'skyview_std_

In [21]:
feature_importances

{'hpmf_raw': 0.003792885687441552,
 'skyview_raw': 0.005059030774157747,
 'impundment_raw': 0.0040016295753498585,
 'slope_raw': 0.005590805775496728,
 'DEM_ditch_detection': 0.001516171508833461,
 'DEM_ditch_detection_no_streams': 0.0018823409000632144,
 'conic_mean': 0.004659880243924091,
 'skyview_non_ditch': 0.014114622472348518,
 'skyview_gabor': 0.013231929263118277,
 'conic_mean_no_streams': 0.004288774479934019,
 'skyview_gabor_no_streams': 0.012756754099458166,
 'skyview_mean_2': 0.005708233914424685,
 'skyview_mean_3': 0.004253462306403661,
 'skyview_mean_4': 0.005345583182336104,
 'skyview_mean_6': 0.006788779175539961,
 'skyview_median_2': 0.006018192813455168,
 'skyview_median_4': 0.00715969571589881,
 'skyview_median_6': 0.011413375997469579,
 'skyview_min_2': 0.005341299330536001,
 'skyview_min_4': 0.00721019511063742,
 'skyview_min_6': 0.009747926269959545,
 'skyview_max_2': 0.008929815728077198,
 'skyview_max_4': 0.011081153023135296,
 'skyview_max_6': 0.01753624376389